### Import necessary libraries

In [ ]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

### Check if GPU is available

In [ ]:
print('Number of GPUs available: ', len(tf.config.experimental.list_physical_devices('GPU')))
print('Test is built with CUDA: ', tf.test.is_built_with_cuda())

### Prepare the dataset

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, shuffle_files=True, split=['train'])

In [ ]:
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    DATASET_SIZE = len(dataset)

    train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

    val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
    val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))
    test_dataset = val_test_dataset.skip(int(TEST_RATIO*DATASET_SIZE))

    return train_dataset, val_dataset, test_dataset

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = split(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

# print(list(train_dataset.take(1).as_numpy_iterator()),
#       list(val_dataset.take(1).as_numpy_iterator()),
#       list(test_dataset.take(1).as_numpy_iterator()))

In [ ]:
for i, (image, label) in enumerate(train_dataset.take(16)):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image)
    plt.title(dataset_info.features['label'].int2str(label))
    plt.axis('off')

### Resize the image

In [ ]:
IM_SIZE = 224

def resize_rescale(image, label):
    return tf.image.resize(image, (IM_SIZE, IM_SIZE)) / 255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

# for image, label in train_dataset.take(1):
#     print(image, label)

### Batch dataset

In [ ]:
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

### Train the model

In [ ]:
model = tf.keras.Sequential([
    InputLayer(shape=(IM_SIZE, IM_SIZE, 3)),
    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),

    Dense(10, activation='relu'),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=BinaryCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=20, verbose=1)

model.save('../models/Malaria.keras')

In [ ]:
model.save_weights('../models/Malaria.weights.h5')

### Plot the history

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()

### Load the model

In [ ]:
loaded_model = tf.keras.models.load_model('../models/Malaria.keras')
loaded_model.summary()

In [ ]:
model.load_weights('../models/Malaria.weights.h5')

### Prepare the evaluation

In [ ]:
def parasite_or_not(x):
    if(x < 0.5):
        return str('P')
    else:
        return str('U')

In [ ]:
test_dataset = test_dataset.batch(1)

### Evaluate the model

In [ ]:
model.evaluate(test_dataset)

In [ ]:
parasite_or_not(model.predict(test_dataset.take(1))[0][0])
model.predict(test_dataset.take(1))[0][0]

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[0])
    plt.title(str(parasite_or_not(label.numpy()[0])) + ': ' + str(parasite_or_not(model.predict(image)[0][0])))
    plt.axis('off')

### Evaluate the LOADED model

In [ ]:
loaded_model.evaluate(test_dataset)

In [ ]:
parasite_or_not(loaded_model.predict(test_dataset.take(1))[0][0])
loaded_model.predict(test_dataset.take(1))[0][0]

In [ ]:
for i, (image, label) in enumerate(test_dataset.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[0])
    plt.title(str(parasite_or_not(label.numpy()[0])) + ": " + str(parasite_or_not(loaded_model.predict(image)[0][0])))
    plt.axis('off')